### 0. Default setting 

In [1]:
import pymysql 
import pandas as pd
import numpy as np 
import re 
import os 
import csv

In [2]:
data_path = os.path.join('/workspace/data/')

In [3]:
args = dict()
args['host'] = "192.168.48.1"
args['user'] = "cori"
args['password'] = "Dhwoehd12!"
args['db'] = "shinhan"
args['port'] = 3306

#### Load data

### 1. 공통 처리 

#### 중복 제거

#### 고유키 생성 (RDB <-> Vector DB 연동용도)

In [198]:
loan = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN.csv'), encoding='utf-8')
card = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD.csv'), encoding='utf-8')
deposit = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT.csv'), encoding='utf-8')

In [199]:
def get_len(tb):
    '''
    테이블 칼럼별 최대 길이 조회
    '''
    max_lens = dict()
    cols = tb.columns
    for col in cols:
        if tb[col].dtype == 'object':
            count_df = tb[col].copy()
            count_df = pd.DataFrame(count_df, columns=[col])
            count_df.dropna(inplace=True)
            max_lens[col] = max(count_df[col].str.len())
    return max_lens 

In [200]:
def replace_space(text):
    '''
    2개 이상의 공백 데이터 -> 1개로 변환 
    '''
    if text is not np.nan:
        txt = ' '.join(text.split())  
        return txt
    else:
        return np.nan

In [201]:
def plot_box(data):
    pass

#### 공백 제거 (2개 이상 -> 1개) 

In [202]:
loan_col = loan.columns.tolist()
card_col = card.columns.tolist()
deposit_col = deposit.columns.tolist()

In [203]:
for col in card_col:
    if card[col].dtype == 'object':
        card[col] = card[col].apply(replace_space)

In [204]:
for col in loan_col:
    if loan[col].dtype == 'object':
        loan[col] = loan[col].apply(replace_space)

In [205]:
for col in deposit_col:
    if deposit[col].dtype == 'object':
        deposit[col] = deposit[col].apply(replace_space)

In [206]:
np.unique(deposit['금융회사코드'].str.len())

array([ 3,  5,  8, 28, 42, 66, 68, 82])

In [207]:
test = loan.copy()
test['금융회사코드'] = test['금융회사코드'].apply(lambda x: str(x))
np.unique(test['금융회사코드'].str.len())

array([3, 4, 5])

In [208]:
test[test['금융회사코드'].str.len() == 4]

,대출_고유키,금융회사코드,금융회사명,금융상품코드,금융상품명,상품종류,대출종류,가입대상,상품특징,대출신청자격,...,기준일자,심의필일자,적재일자,상품설명,유의사항,금리인하,대출철회,이자부과시기,위법계약,대출금리
106,대출_106,34.0,광주은행,LON20210716001,ESG SUPPORT LOAN,기업대출,기업대출,기업,NaN,당행 기업신용등급 BB(소매익스포져인 경우 AS 6등급) 이상으로 아래에 해당하는 ...,...,NaN,준법감시인 심의필 제 2023-4371호 (2023.12.21.),20240104,"ESG(환경,사회,지배구조)경영 강화를 통한 기업상생협력을 위한 금융상품","은행연합회 「여신심사 선진화를 위한 모범규준」 등에 따라 대출가능여부, 대출금액등에...",본 상품은 ＇금리인하요구권＇ 신청이 가능한 상품입니다.금리인하요구권 안내채무자는 본...,NaN,NaN,NaN,기준금리 + 가산금리 - 우대금리
107,대출_107,34.0,광주은행,LON20200121001,KJB SOLAR-LOAN,기업대출,기업대출,개인사업자법인,NaN,발전용량 1MWh 이하의 태양광발전사업 허가와 각종 행정 인 허가를 득한 광주은행 ...,...,NaN,준법감시인 심의필 제 2023-4368호 (2023.12.21.),20240104,태양광발전사업자를 위한 맞춤대출,"은행연합회 「여신심사 선진화를 위한 모범규준」 등에 따라 대출가능여부, 대출금액 등...",본 상품은 ＇금리인하요구권＇ 신청이 가능한 상품입니다.금리인하요구권 안내채무자는 본...,NaN,NaN,NaN,기준금리 + 가산금리 - 우대금리
108,대출_108,34.0,광주은행,LON20230518001,KJB Trust-On 보증서대출,기업대출,기업대출,개인사업자법인,NaN,다음 조건을 모두 충족하는 중소기업을 대상으로 함신용보증기금 「트러스트온(Trust...,...,NaN,준법감시인 심의필 제 2023-1491호 (2023.05.09.),20240104,신용보증기금과 연계한 보증서 담보대출 전용상품으로 대출금 사용의 투명성 확보를 위해...,상기 내용은 신용보증기금 및 광주은행 정책변경에 따라 달라질 수 있습니다.신용보증기...,본 상품은 ＇금리인하요구권＇ 신청이 가능한 상품입니다.금리인하요구권 안내채무자는 본...,NaN,NaN,NaN,기준금리 + 가산금리 - 우대금리
109,대출_109,34.0,광주은행,LON20221219001,KJB모바일 아파트대출,주택관련대출,주택관련대출,실명개인,NaN,"본인 명의(또는 배우자와 공동명의) 아파트를 담보로 제공하고, 전자적 방식을 통해 ...",...,NaN,준법감시인 심의필 제2023-3418호 (2023.10.17),20240104,사전한도 조회 후 신청 가능한 비대면전용 아파트대출,"은행연합회 「여신심사 선진화를 위한 모범규준」 등에 따라 대출가능여부, 대출금액 등...",본 상품은 ＇금리인하요구권&rsquo 신청이 가능한 상품입니다.금리인하요구권 안내채...,NaN,NaN,NaN,기준금리 + 가산금리 - 우대금리
110,대출_110,34.0,광주은행,LON20230718002,KJB모바일 전세대출(서울보증보험),주택관련대출,주택관련대출,서울보증보험 개인금융신용보험이 청약 가능한 고객으로 일정의 자격기준을 만족하는 고객,NaN,전자방식 서류제출서비스를 통해 소득이 확인되는 근로소득자(현 직장 1년 이상 재직)...,...,NaN,준법감시인 심의필 제 2023-3731호 (2023.11.16.),20240104,.,"은행연합회 「여신심사 선진화를 위한 모범규준」 등에 따라 대출가능여부, 대출금액 등...",본 상품은 ＇금리인하요구권&rsquo 신청이 불가능한 상품입니다.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,대출_1418,88.0,신한은행,711120099,소상공인2차금융지원대출(수탁보증)(2021.12.31판매중지),기타대출,NaN,"코로나19피해로인하여어려움을겪고있는개인사업자단,아래①~⑩중어느하나에해당하는경우는제외...",NaN,NaN,...,20240104,준법감시인사전심사필제2023-40734-1호(2023.01.31~2024.01.27),20240104,코로나19피해소상공인여러분에게신용보증기금보증서담보로자금을지원해드립니다.,이금융상품을가입(계약)하시기전에＇금융상품설명서＇및＇약관＇을반드시읽어보시기바랍니다.이...,NaN,NaN,NaN,NaN,금융채(1년):3.7055금융채(1년):1.72
1419,대출_1419,88.0,신한은행,612511300,퇴직연금근로자우대대출(2017.02.24판매중지),신용대출,NaN,퇴직연금을당행에적립하는기업의재직기간1년이상인근로자,NaN,NaN,...,20240104,준법감시인사전심사필제2020-11453-3호(2020.11.02~2021.10.31),20240104,퇴직연금을신한은행에적립하는기업애재직중인고객을위한특별한대출서비스,대출취급시인지세등의부대비용과약정한기일이전에대출금을상환하는경우중도상환수수료가발생될수있...,"채무자는본인의신용등급상승(예.소득유형(근로,사업소득)변경,소득·자산증가,부채감소등)...","본상품은「은행여신거래기본약관(가계용)」제4조2대출계약철회에따라서면,전화,인터넷뱅킹을...",계산방법:1년을365일(윤년은366일)로보고1일단위로계산납입시기-건별대출:이자납입일...,NaN,금융채(6개월):3.8400금융채(1년):3.7100금융채(6개월):2.3125금융...
1420,대출_1420,88.0,신한은행,711117852,코로나19긴급유동성지원특례보증이차보전(2021.01.16판매중지),기타대출,NaN,지역신용보증재단으로부터「코로나19긴급유동성지원」특례보증서를발급받은소상공인지역신용보증...,NaN,NaN,...,20240104,준법감시인사전심사필제2021-40249-2호(2021.09.29~2022.09.29),20240104,코로나19재확산에따른매출급감으로경영안정이시급한소상공인에게지원해드립니다.★한도소진시지...,"대출취급시,고객님부담으로인지세(50%)의부대비용이발생할수있습니다.대출기간이끝났음에도...","본상품은금리인하요구권적용대상이나,이차보전지원제외사유발생하여이차보전율만큼대출금리가인상...",NaN,NaN,NaN,고정2.00%
1421,대출_1421,88.0,신한은행,711117851,소상공인시장진흥공단이관물량처리이차보전대출(2020.12.31판매중지),신용대출,NaN,지역신용보증재단으로부터「코로나19피해소상공인(소상공인정책자금전환전용)」특례보증서를발...,NaN,NaN,...,20240104,준법감시인사전심사필제2022-40357-5호(2022.03.17~2023.03.17),20240104,소상공인정책자금재원부족으로자금집행이불가능한소상공인여러분에게전환상품으로지원해드립니다....,이금융상품을가입(계약)하시기전에＇금융상품설명서＇및＇약관＇을반드시읽어보시기바랍니다.이...,"본상품은금리인하요구권적용대상이나,이차보전지원제외사유발생하여이차보전율만큼대출금리가인상...",NaN,NaN,NaN,고정1.50%


In [209]:
deposit[deposit['금융회사코드'].str.len() == 5]

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,가입방법,이자지급방법,예금자보호,...,저축금리,최고금리,최고 우대금리,가입금액,기준일자,심의필일자,적재일자,상품설명,중도해지금리,상품판매기간
1217,보증_1217,</ol>,"계좌에 압류, 가압류, 질권설정 등이 등록될 경우 원금 및 이자 지급 제한 질권 설...",NaN,"1년, 2년, 3년(연단위)",6.0%,6.000000,NaN,1만원 이상 ~ 10만원 이하,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Pre-process Card data

In [210]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   카드_고유키  1133 non-null   object
 1   카드회사코드  1133 non-null   int64 
 2   카드회사명   1133 non-null   object
 3   상품유형    1083 non-null   object
 4   상품코드    1130 non-null   object
 5   상품명     1129 non-null   object
 6   카드설명    1023 non-null   object
 7   가입대상    53 non-null     object
 8   혜택유형    857 non-null    object
 9   연회비     857 non-null    object
 10  브랜드     1035 non-null   object
 11  카드등급    20 non-null     object
 12  후불교통    940 non-null    object
 13  할인혜택    1057 non-null   object
 14  가입방법    72 non-null     object
 15  발급안내    87 non-null     object
 16  연체이자율   692 non-null    object
 17  유의사항    633 non-null    object
 18  카드이미지   1133 non-null   object
 19  심의필일자   668 non-null    object
 20  적재일자    1133 non-null   int64 
dtypes: int64(2), object(19)
memory usage: 186.0+ KB


In [211]:
card_dict = get_len(card)
card_dict

{'카드_고유키': 7,
 '카드회사명': 7,
 '상품유형': 15,
 '상품코드': 14,
 '상품명': 37,
 '카드설명': 121,
 '가입대상': 110,
 '혜택유형': 347,
 '연회비': 7358,
 '브랜드': 273,
 '카드등급': 7,
 '후불교통': 1,
 '할인혜택': 8468,
 '가입방법': 147,
 '발급안내': 766,
 '연체이자율': 722,
 '유의사항': 8985,
 '카드이미지': 39,
 '심의필일자': 108}

In [212]:
col = '할인혜택'

In [213]:
idx = card[card[col].str.len() == card_dict[col]][col].index.values[0]
idx

121

In [214]:
card.loc[idx][col][:40]

'*기타 *3,000 won discount on TOEIC test fe'

### 3. Pre-process Deposit data

In [215]:
deposit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   보증_고유키   1553 non-null   object 
 1   금융회사코드   1553 non-null   object 
 2   금융회사명    1547 non-null   object 
 3   상품종류     1326 non-null   object 
 4   상품유형     762 non-null    object 
 5   금융상품코드   1152 non-null   object 
 6   금융상품명    1544 non-null   object 
 7   가입방법     1153 non-null   object 
 8   이자지급방법   1168 non-null   object 
 9   예금자보호    1211 non-null   object 
 10  세제혜택     909 non-null    object 
 11  만기후이자율   764 non-null    object 
 12  우대조건     600 non-null    object 
 13  가입제한     164 non-null    object 
 14  가입대상     1436 non-null   object 
 15  기타 유의사항  1252 non-null   object 
 16  최고한도     136 non-null    object 
 17  저축기간     1266 non-null   object 
 18  저축금리     1242 non-null   object 
 19  최고금리     836 non-null    object 
 20  최고 우대금리  344 non-null    object 
 21  가입금액     1228 

In [216]:
deposit.head(1)

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,가입방법,이자지급방법,예금자보호,...,저축금리,최고금리,최고 우대금리,가입금액,기준일자,심의필일자,적재일자,상품설명,중도해지금리,상품판매기간
0,보증_0,002,KDB산업은행,목돈모으기,정기적금,100239000101,KDB 기업정기적금,"영업점, 기업인터넷뱅킹",매월 약정한 날짜에 월저축금을 입금하는 경우- 납입건별(세전 만기) : 납입금액 ×...,"이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호한도는 본 은행에 있는 ...",...,"3.13(6개월 이상),3.1(12개월 이상),2.99(24개월 이상),2.98(3...",3.13,0.0,1만원 이상,20240104,준법감시인 심의필 제2023-10-018(2023.10.27),20240104.0,기업 전용 정기적금 상품,NaN,NaN


In [217]:
deposit_dict = get_len(deposit)
deposit_dict

{'보증_고유키': 7,
 '금융회사코드': 82,
 '금융회사명': 103,
 '상품종류': 102,
 '상품유형': 147,
 '금융상품코드': 23,
 '금융상품명': 38,
 '가입방법': 314,
 '이자지급방법': 615,
 '예금자보호': 692,
 '세제혜택': 974,
 '만기후이자율': 2200,
 '우대조건': 1532,
 '가입제한': 361,
 '가입대상': 569,
 '기타 유의사항': 4819,
 '최고한도': 180,
 '저축기간': 206,
 '저축금리': 1951,
 '최고금리': 132,
 '최고 우대금리': 1102,
 '가입금액': 251,
 '기준일자': 733,
 '심의필일자': 111,
 '상품설명': 378,
 '중도해지금리': 845}

#### 이상치 데이터 제거 

In [218]:
def get_outliers(df, col):
    tmp = pd.DataFrame(deposit[col].copy(), columns=[col])
    tmp['len'] = df[col].str.len()
    len_q3 = tmp['len'].quantile(0.9)
    len_q1 = tmp['len'].quantile(0.1)
    # print(len_q3, len_q1)
    len_iqr = len_q3 - len_q1  
    len_outlier = len_q3 + 1.5 * len_iqr
    # print(f'이상치: {len_outlier}')
    outlier_idx = tmp[tmp[col].str.len() > len_outlier].index.tolist()
    # print(len(outlier_idx))
    return outlier_idx 

In [219]:
len(deposit)

1553

In [220]:
outlier_idxs = []
col_list = ['금융회사코드', '금융회사명']
for col in col_list:
    if deposit[col].dtype == 'object':
        # print(col)
        # print(deposit.loc[get_outliers(deposit, col)][col])
        print(get_outliers(deposit, col))
        outlier_idxs.extend(get_outliers(deposit, col))

[1010, 1212, 1213, 1214, 1215, 1216, 1217, 1218]
[1217]


In [221]:
np.unique([deposit['심의필일자'].str.len()])

array([  9.,  11.,  16.,  20.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,
        29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,
        40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,
        51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,
        62.,  63.,  64.,  65.,  66.,  67.,  69.,  70.,  71.,  72.,  73.,
        93.,  94.,  95.,  96.,  97.,  98.,  99., 100., 101., 102., 109.,
       110., 111.,  nan])

In [222]:
deposit[deposit['심의필일자'].str.len() > 100].head(1)

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,가입방법,이자지급방법,예금자보호,...,저축금리,최고금리,최고 우대금리,가입금액,기준일자,심의필일자,적재일자,상품설명,중도해지금리,상품판매기간
160,보증_160,007,수협,예금,입출금이자유로운예금,D00151,좋은수!Sh주거래기업통장,신규·해지:영업점,NaN,"이예금은예금자보호법에따라예금보험공사가보호하되,보호한도는본은행에있는귀하의모든예금보호대...",...,0.10,NaN,NaN,입출금이자유로운예금,20240104,심의번호:준법감시인심의필상제2023-0947호(2023.03.20.기준)유효기간:2...,20240104.0,거래이용실적에따라수수료우대혜택을제공하는기업전용입출금이자유로운통장,NaN,NaN


In [223]:
deposit.loc[0]

보증_고유키                                                  보증_0
금융회사코드                                                   002
금융회사명                                                KDB산업은행
상품종류                                                   목돈모으기
상품유형                                                    정기적금
금융상품코드                                          100239000101
금융상품명                                             KDB 기업정기적금
가입방법                                            영업점, 기업인터넷뱅킹
이자지급방법     매월 약정한 날짜에 월저축금을 입금하는 경우- 납입건별(세전 만기) : 납입금액 ×...
예금자보호      이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호한도는 본 은행에 있는 ...
세제혜택                                                      가능
만기후이자율                                                   NaN
우대조건                                                      없음
가입제한                                              계좌 수 제한 없음
가입대상                                      법인(임의단체 제외), 개인사업자
기타 유의사항    계약체결 전 상품설명서, 약관을 반드시 읽어보시기 바랍니다. 금융소비자는 해당 상품...
최고한도                    

In [224]:
deposit.loc[160]['심의필일자']

'심의번호:준법감시인심의필상제2023-0947호(2023.03.20.기준)유효기간:2023.03.20.~2025.02.28.(단,기존은행상품의거래조건변경시변경된거래조건시행일의전일까지)'

In [225]:
outlier_idxs = list(set(outlier_idxs))
outlier_idxs

[1216, 1217, 1218, 1010, 1212, 1213, 1214, 1215]

In [226]:
print(len(deposit))
deposit.drop(outlier_idxs, inplace=True)
print(len(deposit))

1553
1545


In [227]:
deposit_dict = get_len(deposit)
deposit_dict

{'보증_고유키': 7,
 '금융회사코드': 3,
 '금융회사명': 10,
 '상품종류': 102,
 '상품유형': 147,
 '금융상품코드': 23,
 '금융상품명': 38,
 '가입방법': 314,
 '이자지급방법': 615,
 '예금자보호': 692,
 '세제혜택': 974,
 '만기후이자율': 2200,
 '우대조건': 1532,
 '가입제한': 361,
 '가입대상': 569,
 '기타 유의사항': 4819,
 '최고한도': 180,
 '저축기간': 206,
 '저축금리': 1951,
 '최고금리': 132,
 '최고 우대금리': 1102,
 '가입금액': 251,
 '기준일자': 733,
 '심의필일자': 111,
 '상품설명': 378,
 '중도해지금리': 845}

#### NULL 값만 존재하는 데이터 제거 

In [228]:
deposit.drop('상품판매기간', axis=1, inplace=True) 

### 4. Pre-process Loan data

In [229]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   대출_고유키     1424 non-null   object 
 1   금융회사코드     1423 non-null   float64
 2   금융회사명      1423 non-null   object 
 3   금융상품코드     1150 non-null   object 
 4   금융상품명      1421 non-null   object 
 5   상품종류       1205 non-null   object 
 6   대출종류       810 non-null    object 
 7   가입대상       1306 non-null   object 
 8   상품특징       867 non-null    object 
 9   대출신청자격     790 non-null    object 
 10  가입방법       958 non-null    object 
 11  대출기간       1343 non-null   object 
 12  상환방법       1275 non-null   object 
 13  대출 부대비용    1232 non-null   object 
 14  중도 상환 수수료  1145 non-null   object 
 15  연체 이자율     1090 non-null   object 
 16  대출한도       1334 non-null   object 
 17  담보 유형      754 non-null    object 
 18  대출 상환유형    582 non-null    object 
 19  대출 금리 유형   292 non-null    object 
 20  우대조건    

In [230]:
loan_dict = get_len(loan)
loan_dict

{'대출_고유키': 7,
 '금융회사명': 10,
 '금융상품코드': 23,
 '금융상품명': 47,
 '상품종류': 89,
 '대출종류': 47,
 '가입대상': 998,
 '상품특징': 752,
 '대출신청자격': 1035,
 '가입방법': 447,
 '대출기간': 989,
 '상환방법': 1354,
 '대출 부대비용': 2675,
 '중도 상환 수수료': 3773,
 '연체 이자율 ': 2161,
 '대출한도': 888,
 '담보 유형': 2064,
 '대출 상환유형': 1190,
 '대출 금리 유형': 3229,
 '우대조건': 2382,
 '최고 우대금리': 275,
 '대출 금리_기준': 862,
 '대출 금리_가산': 735,
 '대출 금리_우대': 2382,
 '대출 금리_최저': 558,
 '대출 금리_최고': 558,
 '기준일자': 316,
 '심의필일자': 1819,
 '상품설명': 416,
 '유의사항': 3032,
 '금리인하': 1673,
 '대출철회': 1473,
 '이자부과시기': 240,
 '위법계약': 1063,
 '대출금리': 3433}

In [231]:
def get_outliers(df, col):
    tmp = pd.DataFrame(df[col].copy(), columns=[col])
    tmp['len'] = df[col].str.len()
    len_q3 = tmp['len'].quantile(0.9)
    len_q1 = tmp['len'].quantile(0.1)
    # print(len_q3, len_q1)
    len_iqr = len_q3 - len_q1  
    len_outlier = len_q3 + 1.5 * len_iqr
    print(f'이상치: {len_outlier}')
    outlier_idx = tmp[tmp[col].str.len() > len_outlier].index.tolist()
    # print(len(outlier_idx))
    return outlier_idx 

In [232]:
outlier_idxs = []
col_list = ['기준일자', '심의필일자']
for col in col_list:
    if loan[col].dtype == 'object':
        # print(col)
        print(get_outliers(loan, col))
        outlier_idxs.extend(get_outliers(loan, col))

이상치: 30.5
[299, 303, 304, 305, 309, 312, 313, 351, 361, 371, 372, 387, 388, 389, 390, 402, 410, 894, 899]
이상치: 30.5
이상치: 103.5
[154, 155, 605, 613, 615, 622, 623, 624, 625, 626, 627, 628, 629, 646, 653, 666, 669, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1287]
이상치: 103.5


In [233]:
print(len(loan))
loan.drop(outlier_idxs, inplace=True)
print(len(loan))

1424
1377


In [234]:
outlier_idxs = list(set(outlier_idxs))
outlier_idxs[:3]

[387, 388, 389]

In [235]:
loan[loan['금융회사코드']==3.0]['금융회사명']

414    IBK기업은행
415    IBK기업은행
416    IBK기업은행
417    IBK기업은행
418    IBK기업은행
        ...   
477    IBK기업은행
478    IBK기업은행
479    IBK기업은행
480    IBK기업은행
481    IBK기업은행
Name: 금융회사명, Length: 68, dtype: object

#### 금융회사코드 변환 

In [236]:
np.unique(loan['기준일자'].str.len())

array([ 6.,  7.,  8.,  9., 10., 13., 14., 17., 19., nan])

In [237]:
loan['금융회사코드'] = loan['금융회사코드'].apply(lambda x: str(x).split('.')[0])

In [238]:
loan['금융회사코드'] = loan['금융회사코드'].apply(lambda x: x.zfill(3))
loan.head(1)

,대출_고유키,금융회사코드,금융회사명,금융상품코드,금융상품명,상품종류,대출종류,가입대상,상품특징,대출신청자격,...,기준일자,심의필일자,적재일자,상품설명,유의사항,금리인하,대출철회,이자부과시기,위법계약,대출금리
0,대출_0,002,KDB산업은행,A100000000,혁신성장산업 지원자금,기업대출,NaN,기업,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원,○ (지원대상) 다음 중 하나에 해당하는 기업 혁신성장 정책금융협의회 선정 「혁신성...,...,NaN,준법감시인 심의필제2023-10-013호(2023.10.27),20240104,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원,* 금융소비자는 해당 상품에 관하여 충분히 설명을 받을 권리가 있습니다.* 대출가능...,NaN,NaN,NaN,NaN,"대출금리 기본 : (원화) CD3개월 유통수익율, 해당만기별 산금채 유통수익율(6개..."


In [239]:
loan_dict = get_len(loan)
loan_dict

{'대출_고유키': 7,
 '금융회사코드': 3,
 '금융회사명': 10,
 '금융상품코드': 23,
 '금융상품명': 47,
 '상품종류': 89,
 '대출종류': 39,
 '가입대상': 998,
 '상품특징': 752,
 '대출신청자격': 1035,
 '가입방법': 416,
 '대출기간': 989,
 '상환방법': 1354,
 '대출 부대비용': 2675,
 '중도 상환 수수료': 3773,
 '연체 이자율 ': 2161,
 '대출한도': 888,
 '담보 유형': 2064,
 '대출 상환유형': 1190,
 '대출 금리 유형': 3229,
 '우대조건': 2382,
 '최고 우대금리': 275,
 '대출 금리_기준': 862,
 '대출 금리_가산': 735,
 '대출 금리_우대': 2382,
 '대출 금리_최저': 558,
 '대출 금리_최고': 558,
 '기준일자': 19,
 '심의필일자': 103,
 '상품설명': 416,
 '유의사항': 3032,
 '금리인하': 1673,
 '대출철회': 1473,
 '이자부과시기': 240,
 '위법계약': 1063,
 '대출금리': 3433}

In [240]:
loan[loan['대출 금리_최고'].str.len() == 558]['대출 금리_최고'].values

array(['변동금리 3M CD 6.08%, 변동금리 COFIX신규 6.25%, 변동금리 COFIX잔액 6.14%, 변동금리 신 잔액기준 COFIX 5.60%, 변동금리 금융채1년 5.99%, 변동금리 금융채3년 5.95%, 변동금리 금융채5년 6.04%, 고정금리 금융채1년 5.99%, 고정금리 금융채2년 5.92%, 고정금리 금융채3년 5.95%, 고정금리 금융채5년 6.04%, 혼합금리 금융채1년 5.99%, 혼합금리 금융채2년 5.92%, 혼합금리 금융채3년 5.95%, 혼합금리 금융채5년 6.04%변동금리 3M CD 6.23%, 변동금리 COFIX신규 6.40%, 변동금리 COFIX잔액 6.29%, 변동금리 신 잔액기준 COFIX 5.75%, 변동금리 금융채1년 6.14%, 변동금리 금융채3년 6.10%, 변동금리 금융채5년 6.19%, 고정금리 금융채1년 6.14%, 고정금리 금융채2년 6.07%, 고정금리 금융채3년 6.10%, 고정금리 금융채5년 6.19%, 혼합금리 금융채1년 6.14%, 혼합금리 금융채2년 6.07%, 혼합금리 금융채3년 6.10%, 혼합금리 금융채5년 6.19%',
       '변동금리 3M CD 6.18%, 변동금리 COFIX신규 6.35%, 변동금리 COFIX잔액 6.24%, 변동금리 신 잔액기준 COFIX 5.70%, 변동금리 금융채1년 6.09%, 변동금리 금융채3년 6.05%, 변동금리 금융채5년 6.14%, 고정금리 금융채1년 6.09%, 고정금리 금융채2년 6.02%, 고정금리 금융채3년 6.05%, 고정금리 금융채5년 6.14%, 혼합금리 금융채1년 6.09%, 혼합금리 금융채2년 6.02%, 혼합금리 금융채3년 6.05%, 혼합금리 금융채5년 6.14%변동금리 3M CD 6.33%, 변동금리 COFIX신규 6.50%, 변동금리 COFIX잔액 6.39%, 변동금리 신 잔액기준 COFIX 5.85%, 변동금리 금융채1년 6.24%, 변동금리 금융채3년 6.20%, 변동금리 금융채5년 6.29%, 고정금리 금융

In [241]:
card.reset_index(inplace=True, drop=True)
loan.reset_index(inplace=True, drop=True)
deposit.reset_index(inplace=True, drop=True)

In [242]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377 entries, 0 to 1376
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   대출_고유키     1377 non-null   object
 1   금융회사코드     1377 non-null   object
 2   금융회사명      1376 non-null   object
 3   금융상품코드     1116 non-null   object
 4   금융상품명      1374 non-null   object
 5   상품종류       1169 non-null   object
 6   대출종류       789 non-null    object
 7   가입대상       1273 non-null   object
 8   상품특징       835 non-null    object
 9   대출신청자격     756 non-null    object
 10  가입방법       924 non-null    object
 11  대출기간       1300 non-null   object
 12  상환방법       1231 non-null   object
 13  대출 부대비용    1195 non-null   object
 14  중도 상환 수수료  1105 non-null   object
 15  연체 이자율     1079 non-null   object
 16  대출한도       1291 non-null   object
 17  담보 유형      729 non-null    object
 18  대출 상환유형    566 non-null    object
 19  대출 금리 유형   279 non-null    object
 20  우대조건       451 non-null    obj

In [243]:
card[card['상품유형'].isna()]

,카드_고유키,카드회사코드,카드회사명,상품유형,상품코드,상품명,카드설명,가입대상,혜택유형,연회비,...,카드등급,후불교통,할인혜택,가입방법,발급안내,연체이자율,유의사항,카드이미지,심의필일자,적재일자
0,카드_0,323,수협은행카드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,N,NaN,NaN,NaN,NaN,NaN,20240104/cardImage_Y/.png,NaN,20240104
1,카드_1,323,수협은행카드,NaN,C00053,휴먼라이프 S1 수협카드,바다위새로운세상휴먼라이프크루즈여행을위한맞춤혜택,NaN,할인형,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00053.png,준법감시인심의필상제2023-2517(2023.07.10.기준),20240104
2,카드_2,323,수협은행카드,NaN,C00006,S1카드,7개업종중선택한1개업종가맹점에서카드사용시7%청구할인,개인회원(가족회원 포함)*수협은행 신용카드 발급기준에 부합하는 자,"할인형, 캐쉬백, 적립형",NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00006.png,준법감시인심의필상제2023-2516(2023.07.10.기준),20240104
4,카드_4,323,수협은행카드,NaN,C00036,국고보조금전용카드(e나라도움카드) (개인),국고보조금집행시사용되는카드,개인,NaN,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",발급받은 카드가 국내외겸용인 경우 브랜드 로고( )가 부착되어 있는 전 세계 가맹점...,20240104/cardImage_Y/C00036.png,준법감시인심의필상제2023-2518(2023.07.10.기준),20240104
7,카드_7,323,수협은행카드,NaN,C00021,비자플래티늄카드 (개인),특별한1%의당신을위해남다른품격과가치의시작,개인,NaN,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00021.png,준법감시인심의필상제2023-4306(2023.12.21.기준),20240104
9,카드_9,323,수협은행카드,NaN,C00022,비즈플러스,성공비즈니스를위한든든한파트너,"개인사업자(단, 법인사업자 제외)",NaN,NaN,...,NaN,N,"모든주유소리터당40원청구할인(LPG포함)경유,등유는휘발유가격으로환산하여할인적용휘발유...",관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00022.png,준법감시인심의필상제2023-2545(2023.07.10.기준),20240104
10,카드_10,323,수협은행카드,NaN,C00040,독도사랑카드 (개인),ILoveDokdo,개인,NaN,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00040.png,준법감시인심의필상제2023-4305(2023.12.21.기준),20240104
13,카드_13,323,수협은행카드,NaN,C00003,Feel카드(개인),"행복한생활,쇼핑의즐거움",개인회원(가족회원 포함)*수협은행 신용카드 발급기준에 부합하는 자,NaN,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00003.png,준법감시인심의필상제2023-4304(2023.12.21.기준),20240104
16,카드_16,323,수협은행카드,NaN,C00019,블루스카이카드 (기업),회원의카드이용금액에따라제휴업체인대한항공의스카이패스마일리지를적립해주는상품,기업,NaN,NaN,...,NaN,N,"스카이패스마일리지적립서비스카드매출액3,000원당1마일적립적립한도제한없음※스카이패스마...",관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00019.png,준법감시인심의필상제2023-2550(2023.07.10.기준),20240104
22,카드_22,323,수협은행카드,NaN,C00030,제주도 보조금카드,제주도청에서민간(개인및단체)에지원되는보조금예산에대한집행을수협은행에서발행된보조금전용카...,NaN,NaN,NaN,...,NaN,N,NaN,관심상품등록,NaN,"회원별, 이용상품별 약정금리+최대 3%(p), 법정 최고금리(20%) 이내 단, 연...",이 안내장은 관계 법령 및 내부통제기준에 따른 절차를 준수 하였습니다.계약체결 전 ...,20240104/cardImage_Y/C00030.png,준법감시인심의필상제2023-2534(2023.07.10.기준),20240104


In [244]:
deposit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1545 entries, 0 to 1544
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   보증_고유키   1545 non-null   object 
 1   금융회사코드   1545 non-null   object 
 2   금융회사명    1545 non-null   object 
 3   상품종류     1325 non-null   object 
 4   상품유형     760 non-null    object 
 5   금융상품코드   1151 non-null   object 
 6   금융상품명    1543 non-null   object 
 7   가입방법     1153 non-null   object 
 8   이자지급방법   1167 non-null   object 
 9   예금자보호    1211 non-null   object 
 10  세제혜택     908 non-null    object 
 11  만기후이자율   763 non-null    object 
 12  우대조건     599 non-null    object 
 13  가입제한     164 non-null    object 
 14  가입대상     1436 non-null   object 
 15  기타 유의사항  1252 non-null   object 
 16  최고한도     136 non-null    object 
 17  저축기간     1266 non-null   object 
 18  저축금리     1242 non-null   object 
 19  최고금리     836 non-null    object 
 20  최고 우대금리  344 non-null    object 
 21  가입금액     1228 

In [247]:
loan.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_f.csv'), index=False)
card.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD_f.csv'), index=False)
deposit.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT_f.csv'), index=False)

### DB, Vector DB Table 생성 

In [248]:
loan = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_f.csv'))
card = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD_f.csv'))
deposit = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT_f.csv'))

In [249]:
deposit.dropna(subset=['금융회사코드'], inplace=True)
deposit.isna().sum()

보증_고유키        0
금융회사코드        0
금융회사명         0
상품종류        220
상품유형        783
금융상품코드      392
금융상품명         0
가입방법        391
이자지급방법      377
예금자보호       333
세제혜택        636
만기후이자율      780
우대조건        944
가입제한       1379
가입대상        109
기타 유의사항     292
최고한도       1407
저축기간        277
저축금리        301
최고금리        707
최고 우대금리    1199
가입금액        315
기준일자        821
심의필일자       130
적재일자          1
상품설명         16
중도해지금리     1175
dtype: int64

In [250]:
deposit['금융회사코드'] = deposit['금융회사코드'].apply(lambda x: str(x).split('.')[0])
deposit['금융회사코드'] = deposit['금융회사코드'].apply(lambda x: str(x).zfill(3))
deposit['금융회사코드'] = deposit['금융회사코드'].astype('str')
# loan['금융회사코드'] = loan['금융회사코드'].apply('="{}"'.format)
deposit.head(1)

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,가입방법,이자지급방법,예금자보호,...,저축기간,저축금리,최고금리,최고 우대금리,가입금액,기준일자,심의필일자,적재일자,상품설명,중도해지금리
0,보증_0,002,KDB산업은행,목돈모으기,정기적금,100239000101,KDB 기업정기적금,"영업점, 기업인터넷뱅킹",매월 약정한 날짜에 월저축금을 입금하는 경우- 납입건별(세전 만기) : 납입금액 ×...,"이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호한도는 본 은행에 있는 ...",...,6개월 이상 3년 이하,"3.13(6개월 이상),3.1(12개월 이상),2.99(24개월 이상),2.98(3...",3.13,0.0,1만원 이상,20240104,준법감시인 심의필 제2023-10-018(2023.10.27),20240104.0,기업 전용 정기적금 상품,NaN


In [273]:
column = '금융상품명'
deposit.dropna(subset=column, axis=0, inplace=True)

In [274]:
loan.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_f.csv'), index=False)
card.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD_f.csv'), index=False)
deposit.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT_f.csv'), index=False)

In [275]:
deposit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1543 entries, 0 to 1542
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   보증_고유키   1543 non-null   object 
 1   금융회사코드   1543 non-null   object 
 2   금융회사명    1543 non-null   object 
 3   상품종류     1323 non-null   object 
 4   상품유형     760 non-null    object 
 5   금융상품코드   1151 non-null   object 
 6   금융상품명    1543 non-null   object 
 7   가입방법     1152 non-null   object 
 8   이자지급방법   1166 non-null   object 
 9   예금자보호    1210 non-null   object 
 10  세제혜택     907 non-null    object 
 11  만기후이자율   763 non-null    object 
 12  우대조건     599 non-null    object 
 13  가입제한     164 non-null    object 
 14  가입대상     1434 non-null   object 
 15  기타 유의사항  1251 non-null   object 
 16  최고한도     136 non-null    object 
 17  저축기간     1266 non-null   object 
 18  저축금리     1242 non-null   object 
 19  최고금리     836 non-null    object 
 20  최고 우대금리  344 non-null    object 
 21  가입금액     1228 

In [276]:
col = ['카드_고유키', '카드회사코드', '카드회사명', '상품유형', '상품코드', '상품명', '카드등급', '후불교통', '카드이미지', '심의필일자', '적재일자']
col2 = ['보증_고유키', '금융회사코드', '금융회사명', '상품종류', '상품유형', '금융상품코드', '금융상품명', '심의필일자', '적재일자']
col3 = ['대출_고유키', '금융회사코드', '금융회사명', '금융상품코드', '금융상품명', '기준일자', '심의필일자', '적재일자']
card_db = card[col]
deposit_db = deposit[col2]
loan_db = loan[col3]

In [277]:
deposit_db['금융회사코드'] = deposit_db['금융회사코드'].apply(lambda x: str(x).split('.')[0])
deposit_db['금융회사코드'] = deposit_db['금융회사코드'].apply(lambda x: str(x).zfill(3))
deposit_db['금융회사코드'] = deposit_db['금융회사코드'].astype('str')
# loan['금융회사코드'] = loan['금융회사코드'].apply('="{}"'.format)
deposit_db.head(1)

/tmp/ipykernel_661/1591191821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deposit_db['금융회사코드'] = deposit_db['금융회사코드'].apply(lambda x: str(x).split('.')[0])
/tmp/ipykernel_661/1591191821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deposit_db['금융회사코드'] = deposit_db['금융회사코드'].apply(lambda x: str(x).zfill(3))
/tmp/ipykernel_661/1591191821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,심의필일자,적재일자
0,보증_0,002,KDB산업은행,목돈모으기,정기적금,100239000101,KDB 기업정기적금,준법감시인 심의필 제2023-10-018(2023.10.27),20240104.0


In [278]:
deposit_db.head(1)

,보증_고유키,금융회사코드,금융회사명,상품종류,상품유형,금융상품코드,금융상품명,심의필일자,적재일자
0,보증_0,002,KDB산업은행,목돈모으기,정기적금,100239000101,KDB 기업정기적금,준법감시인 심의필 제2023-10-018(2023.10.27),20240104.0


In [279]:
loan_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_db.csv'), index=False)
card_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD_db.csv'), index=False)
deposit_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT_db.csv'), index=False)

In [280]:
deposit.columns.tolist()

['보증_고유키',
 '금융회사코드',
 '금융회사명',
 '상품종류',
 '상품유형',
 '금융상품코드',
 '금융상품명',
 '가입방법',
 '이자지급방법',
 '예금자보호',
 '세제혜택',
 '만기후이자율',
 '우대조건',
 '가입제한',
 '가입대상',
 '기타 유의사항',
 '최고한도',
 '저축기간',
 '저축금리',
 '최고금리',
 '최고 우대금리',
 '가입금액',
 '기준일자',
 '심의필일자',
 '적재일자',
 '상품설명',
 '중도해지금리']

In [281]:
vector_card = ['카드회사코드', '상품유형', '상품코드', '카드등급', '후불교통', '카드이미지', '심의필일자', '적재일자']
vector_loan = ['금융회사코드', '금융상품코드', '기준일자', '심의필일자', '적재일자']
vector_deposit = ['금융회사코드', '상품종류', '상품유형', '금융상품코드', '심의필일자', '적재일자']

In [282]:
vector_card_db = card.drop(vector_card, axis=1)
vector_loan_db = loan.drop(vector_loan, axis=1)
vector_deposit_db = deposit.drop(vector_deposit, axis=1)

In [285]:
vector_deposit_db.isna().sum()

보증_고유키        0
금융회사명         0
금융상품명         0
가입방법        391
이자지급방법      377
예금자보호       333
세제혜택        636
만기후이자율      780
우대조건        944
가입제한       1379
가입대상        109
기타 유의사항     292
최고한도       1407
저축기간        277
저축금리        301
최고금리        707
최고 우대금리    1199
가입금액        315
기준일자        821
상품설명         16
중도해지금리     1175
dtype: int64

In [286]:
vector_loan_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_vec.csv'), index=False)
vector_card_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_CARD_vec.csv'), index=False)
vector_deposit_db.to_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_DEPOSIT_vec.csv'), index=False)